In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

from preprocessing import preproc
from tools import log_loss_lgbm

from datetime import datetime
import time

path_to_data = "/home/raph/Downloads/"

print("loading data")
training = pd.read_csv(path_to_data+"training.csv")
testing = pd.read_csv(path_to_data+"testing.csv")

loading data


In [2]:
path_to_data = "/home/raph/Downloads/"
to_submit = pd.read_csv(path_to_data+"training.csv")

In [3]:
print("changing dates to time stamps")
training["membership_expire_date"] = training.membership_expire_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
training["membership_expire_date"] = training.membership_expire_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

training["transaction_date"] = training.transaction_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
training["transaction_date"] = training.transaction_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

training["registration_init_time"] = training.registration_init_time.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
training["registration_init_time"] = training.registration_init_time.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

testing["membership_expire_date"] = testing.membership_expire_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
testing["membership_expire_date"] = testing.membership_expire_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

testing["transaction_date"] = testing.transaction_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
testing["transaction_date"] = testing.transaction_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

testing["registration_init_time"] = testing.registration_init_time.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
testing["registration_init_time"] = testing.registration_init_time.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

changing dates to time stamps


In [4]:
training.describe()

,actual_amount_paid,bd,city,gender,is_auto_renew,is_cancel,is_churn,membership_expire_date,payment_method_id,payment_plan_days,plan_list_price,registered_via,registration_init_time,total_number_of_transactions,transaction_date,usual_price_per_day,price_per_day,price_per_day_diff
count,1.074180e+06,1.082190e+06,1.082190e+06,1.082190e+06,1.074180e+06,1.074180e+06,1.082190e+06,1.082190e+06,1.074180e+06,1.074180e+06,1.074180e+06,1.082190e+06,1.082190e+06,1.082190e+06,1.082190e+06,1.082190e+06,1.082190e+06,1.082190e+06
mean,1.636088e+02,1.147173e+01,5.223976e+00,-3.980576e-01,8.594779e-01,1.896610e-02,1.080938e-01,1.480507e+09,3.865341e+01,3.824244e+01,1.639574e+02,5.978358e+00,1.331533e+09,1.263908e+01,1.476861e+09,2.689835e+02,1.463547e+01,-2.543480e+02
std,2.047285e+02,1.609724e+01,6.474496e+00,7.915197e-01,3.475280e-01,1.364053e-01,3.104990e-01,1.281917e+08,4.072921e+00,4.786524e+01,2.045400e+02,3.098580e+00,1.627113e+08,7.486587e+00,1.277328e+08,1.061106e+03,6.346716e+02,1.096345e+03
min,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.600000e+03,3.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,9.466812e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.770237e+05
25%,9.900000e+01,-1.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.491430e+09,3.800000e+01,3.000000e+01,9.900000e+01,4.000000e+00,1.291763e+09,6.000000e+00,1.488496e+09,1.997780e+00,3.298900e+00,3.619256e-01
50%,1.490000e+02,-1.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.492207e+09,4.100000e+01,3.000000e+01,1.490000e+02,7.000000e+00,1.386371e+09,1.300000e+01,1.489273e+09,2.748282e+00,4.965012e+00,1.302228e+00
75%,1.490000e+02,2.600000e+01,1.100000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.492985e+09,4.100000e+01,3.000000e+01,1.490000e+02,9.000000e+00,1.449702e+09,1.900000e+01,1.490137e+09,3.960396e+00,4.965012e+00,2.033633e+00
max,2.000000e+03,9.900000e+01,2.200000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.712700e+09,4.100000e+01,4.500000e+02,2.000000e+03,1.300000e+01,1.492985e+09,2.380000e+02,1.490911e+09,1.770297e+05,1.788000e+05,1.527562e+05


In [5]:
print("preprocessing")
X_train, y_train = preproc(training, mode='train', oneHot=False)
X_test, y_test = preproc(testing, mode="test", oneHot=False)

# parameters
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'binary_logloss'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

i = 0
K = 5
kf = KFold(n_splits=K, random_state=42, shuffle=True)

# training with KFold Cross Validation
results = []
weights = np.zeros(len(y_train))
weights[y_train == 0] = 1
weights[y_train == 1] = 30

preprocessing


In [6]:
print('Start training...')
for train_index, test_index in kf.split(X_train):
    lgb_train = lgb.Dataset(X_train[train_index], y_train[train_index], weight=weights[train_index])
    lgb_eval = lgb.Dataset(X_train[test_index], y_train[test_index], reference=lgb_train)
    gbm = lgb.train(params,
        train_set=lgb_train,
        num_boost_round=200,
        valid_sets=lgb_eval,
        early_stopping_rounds=10,
        verbose_eval=5,
        feval=log_loss_lgbm)
    res = gbm.predict(X_test)
    i += 1
    results.append(res)

Start training...
Training until validation scores don't improve for 10 rounds.
[5]	valid_0's binary_logloss: 0.266212	valid_0's log loss: 0.266212
[10]	valid_0's binary_logloss: 0.257853	valid_0's log loss: 0.257853
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.314073	valid_0's log loss: 0.314073
Training until validation scores don't improve for 10 rounds.
[5]	valid_0's binary_logloss: 0.266765	valid_0's log loss: 0.266765
[10]	valid_0's binary_logloss: 0.258101	valid_0's log loss: 0.258101
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.315297	valid_0's log loss: 0.315297
Training until validation scores don't improve for 10 rounds.
[5]	valid_0's binary_logloss: 0.2661	valid_0's log loss: 0.2661
[10]	valid_0's binary_logloss: 0.257499	valid_0's log loss: 0.257499
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.313918	valid_0's log loss: 0.313918
Training until validation scores don't improve for 10 rounds.
[5]	valid_0's bina

In [ ]:
submission = pd.DataFrame((results[0] + results[1] + results[2] + results[3] + results[4]) / 5)

In [ ]:
submission.columns = ["is_churn"]

In [ ]:
submission.describe()

In [ ]:
submission["msno"] = testing["msno"]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('5Kfold_lgbm.csv', header=True, index=False)
print("created submission file")

In [ ]:
submission.describe()